In [ ]:
import numpy as np
# import Decoder as dec
# from Analysis import *
# import Analysis as anl
import matplotlib.pyplot as plt
# import Dataset
# import Training as Train
# import Optimizing as Opt
import profile
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression  
from sklearn.metrics import confusion_matrix
from matplotlib.colors import ListedColormap


In [ ]:
def loadFile(fileType, m = 0, n = 0):      

    fileName = '/home/jgould/Downloads/temp/Monkey' + fileType + '.mat'
    data = scipy.io.loadmat(fileName)

    Output = np.transpose(data.get('y')) #Kinematic Data
    Input = np.transpose(data.get('X')) # Neural Data
    targs = np.transpose(data.get('targ')) # Target locations
    targsize = data.get('targsize')[0]
    targsize = targsize[0]
    trials = np.transpose(data.get('trials'))[0]
    
    if n == 0:
        n = np.shape(Output)[1]
        
    if m > n:
        print("Error, start index must be lower than end index")

    # trialInfo = self.getTrialInfo(trials[m:n], targs[:, m:n])

    Output, Input = formatXY(Output[:, m:n], Input[:, m:n])
    
    X = Output
    Y = Input
    
    return X, Y

In [ ]:
NMC = Dataset.MonkeyData('N_MC', [1])
WMC = Dataset.MonkeyData('W_MC', [1])
NN = NMC.Y.shape[1]
NW = WMC.Y.shape[1]

YNMC = NMC.Y[:, :int(NN/2)]
YWMC = WMC.Y[:, :int(NW/2)]

jointData = np.append(YNMC, YWMC, axis = 1)


In [ ]:
# from bw_run import AnimationManager
# from bw_run import BWRun
import adaptive_latents.plotting_functions as bpf
from adaptive_latents.regressions import NearestNeighborRegressor

from adaptive_latents import default_rwd_parameters, Bubblewrap, SymmetricNoisyRegressor
from adaptive_latents.bw_run import AnimationManager, BWRun
# from bw_run import BWRun
from  adaptive_latents.plotting_functions import *
import adaptive_latents
import adaptive_latents.input_sources.functional as fin
# import tqdm as tqdm
from adaptive_latents.input_sources.data_sources import NumpyTimedDataSource

def main(Data, steps_to_run=None, nBubbles=200, nDims=96, svd = False):
    Y = Data.Y
    # if svd == True:
    #     prosvd_init = nDims
    #     psvd = proSVD(nDims)
    #     psvd.initialize(Y[:, :prosvd_init].T)

    # print(psvd)

    # gg @ psvd.Q 

    obs = (Data.Y).T
    beh = (Data.X).T
    in_ds = NumpyTimedDataSource(obs, None, time_offsets=(1,))
    out_ds = NumpyTimedDataSource(beh, None, time_offsets=(1,))

    if in_ds.output_shape != nDims:
        print("SOMETHIGN WENT HORRIBLY WRONG")
        print(in_ds.output_shape)
        
    # define the adaptive_latents object
    bw = Bubblewrap(dim=nDims,  **dict(default_rwd_parameters, num = nBubbles, B_thresh=-15, copy_row_on_teleport=False))
    # bw.N = 10

    # define the (optional) method to regress the HMM state from `bw.alpha`
    # reg = SymmetricNoisyRegressor(input_d=bw.N, output_d=1)
    # reg = NearestNeighborRegressor(input_d=bw.N, output_d=1, maxlen=600)

    class CustomAnimation(AnimationManager):
        n_rows = 1
        n_cols = 1
        figsize = (15,10)
        extension = "mp4"

        def custom_draw_frame(self, step, bw: Bubblewrap, br: BWRun):
            historical_observations = br.obs_ds.get_history()

            bpf.show_bubbles_2d(self.ax[0,0], historical_observations, bw, alpha_coefficient=.5)
            self.ax[0,0].set_title(f"Step {step}")
        def frame_draw_condition(self, step_number, bw):
            return step_number % 500 == 0
    # am = CustomAnimation()

    # define the object to coordinate all the other objects
    br = BWRun(bw=bw, obs_ds=in_ds, beh_ds=out_ds, show_tqdm=True, steps = steps_to_run, dims=nDims)

    # run and save the output
    br.run(limit=steps_to_run, save = True)
    return br.pickle_file




In [ ]:
from proSVD import proSVD
def reducePSVD(fullData, ncomp):
    X = fullData.T

    print(np.shape(X))
    pro = proSVD(k=ncomp)
    pro.initialize(X[:100, :].T)

    output = []
    for i in range(100, np.shape(X)[0]):
        obs = X[i:i + 1, :]
        if np.any(np.isnan(obs)):
            output.append(np.zeros(ncomp) * np.nan)
            continue
        pro.preupdate()
        pro.updateSVD(obs.T)
        pro.postupdate()

        obs = obs @ pro.Q

        output.append(obs)
        # reduced = DataSet(fullData.X[:, 100:], np.array(output).reshape((-1, ncomp)).T, fullData)
    return output


In [ ]:
reduced = reducePSVD(jointData, 10)


In [ ]:
br = main(reduced, nBubbles = 100, nDims = 10)